In [1]:
import json
import yfinance as yf
from coskweness_cokurtosis import coskewness, cokurtosis
from portfolio_hubo_qaoa_light import HigherOrderPortfolioQAOA
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import sample_cov
import pennylane as qml
import qiskit
from qiskit import qasm2
from qiskit.transpiler.passes import ResourceEstimation


# Load experiments data
experiments = None
with open("experiments_data.json", "r") as f:
    experiments = list(json.load(f)["data"])
    
print(experiments[0])

circuit_specs = {}
with open("circuit_specs.json", "r") as f:
    circuit_specs = json.load(f)

print(len(circuit_specs))

{'max_qubits': 10, 'budget': 3670, 'stocks': ['HON', 'JNJ'], 'start': '2015-01-01', 'end': '2025-01-01', 'n_stocks': 2}
1


In [2]:
start_index = 0
for typ in ["QUBO"]:
    if typ not in circuit_specs:
        circuit_specs[typ] = {}
    
    for i, experiment in enumerate(experiments[start_index:]):
        i += start_index
        circuit_specs[typ][str(i)] = {}
        stocks = experiment["stocks"]
        start = experiment["start"]
        end = experiment["end"]
        risk_aversion = 0.1
        max_qubits = 15
        budget = experiment["budget"]

        data = yf.download(stocks, start=start, end=end)
        prices_now = data["Close"].iloc[-1]
        returns = data["Close"].pct_change(fill_method=None).dropna(how="any")
        stocks = returns.columns
        numpy_returns = returns.to_numpy()

        expected_returns = mean_historical_return(returns, returns_data=True).to_numpy()
        covariance_matrix = sample_cov(returns, returns_data=True).to_numpy()
        if typ == "QUBO":
            coskewness_tensor = None
            cokurtosis_tensor = None
        else:
            coskewness_tensor = coskewness(numpy_returns)
            cokurtosis_tensor = cokurtosis(numpy_returns)

        portfolio_hubo = HigherOrderPortfolioQAOA(stocks=stocks,
                                                prices_now=prices_now,
                                                expected_returns=expected_returns, 
                                                covariance_matrix=covariance_matrix,
                                                budget=budget,
                                                max_qubits=max_qubits,
                                                coskewness_tensor=coskewness_tensor, 
                                                cokurtosis_tensor=cokurtosis_tensor,
                                                log_encoding=True,
                                                risk_aversion=risk_aversion,
                                                strict_budget_constraint=False)
        n_qubits = portfolio_hubo.get_n_qubits()
        circuit, probs_circuit = portfolio_hubo.get_compiled_QAOA_circuits()
        #circuit.construct([portfolio_hubo.get_init_params()], {})
        print(f"circuit {i} constructed")
        #s = qml.specs(circuit)()
        qtape = qml.workflow.construct_tape(circuit)(portfolio_hubo.get_init_params())
        qasm_str = qtape.to_openqasm(rotations=False)
        print(f"circuit {i} converted to qasm")
        qiskit_circuit = qasm2.loads(qasm_str)
        print(f"circuit {i} converted to qiskit")
        #res_estimation = ResourceEstimation().run(qiskit_circuit)
        gate_types = qiskit_circuit.count_ops()  # Get the count of each gate type

        circuit_specs_alt = {
            "gate_types": dict(gate_types),
            "depth": qiskit_circuit.depth(),
            "num_wires": qiskit_circuit.num_qubits,
            "num_gates": qiskit_circuit.size(),
            "num_params": sum(len(op.operation.params) for op in qiskit_circuit.data)  # Count trainable parameters
        }
        circuit_specs[typ][str(i)]["specs"] = circuit_specs_alt

        with open("circuit_specs.json", "w") as f:
            json.dump(circuit_specs, f, indent=4)
            
        print("Circuit specs saved in circuit_specs.json")
        

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed


Number of qubits for asset HON: 5
Number of qubits for asset JNJ: 5
Total number of qubits:  10
circuit 0 constructed
circuit 0 converted to qasm
circuit 0 converted to qiskit
Circuit specs saved in circuit_specs.json


[*********************100%***********************]  2 of 2 completed


Number of qubits for asset NKE: 6
Number of qubits for asset TRV: 4
Total number of qubits:  10
circuit 1 constructed


...

Circuit specs saved in circuit_specs.json
Number of qubits for asset MCD: 3
Number of qubits for asset TRV: 3
Total number of qubits:  6
circuit 97 constructed



[*********************100%***********************]  2 of 2 completed


circuit 97 converted to qasm
circuit 97 converted to qiskit
Circuit specs saved in circuit_specs.json
Number of qubits for asset AXP: 3
Number of qubits for asset HD: 3
Total number of qubits:  6
circuit 98 constructed
circuit 98 converted to qasm
circuit 98 converted to qiskit
Circuit specs saved in circuit_specs.json


[*********************100%***********************]  3 of 3 completed

Number of qubits for asset AAPL: 2
Number of qubits for asset AMGN: 2
Number of qubits for asset AXP: 2
Total number of qubits:  6
circuit 99 constructed
circuit 99 converted to qasm
circuit 99 converted to qiskit
Circuit specs saved in circuit_specs.json


In [3]:
import pennylane as qml

# Define a PennyLane device (doesn't have to be an actual backend for QASM export)
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RX(0.3, wires=1)
    return qml.probs(wires=[0, 1])


circuit.construct([], {})  # Construct the circuit without execution

qasm_str = circuit.qtape.to_openqasm()
print(qasm_str)


OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
rx(0.3) q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];



c:\Users\valte\OneDrive - University of Helsinki\Desktop\quantum-portfolio\venv\lib\site-packages\pennylane\workflow\qnode.py:830: PennyLaneDeprecationWarning: The tape/qtape property is deprecated and will be removed in v0.41. Instead, use the qml.workflow.construct_tape function.
  warnings.warn(
